## Session 7 Homework

Introduction to BentoML

In [5]:
import bentoml
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb

In [4]:
!bentoml --version

bentoml, version 1.0.7


In [6]:
data = 'CreditScoring.csv'
df = pd.read_csv(data)

In [7]:
df.columns = df.columns.str.lower()

status_values = {
    1: 'ok',
    2: 'default',
    0: 'unk'
}

df.status = df.status.map(status_values)

home_values = {
    1: 'rent',
    2: 'owner',
    3: 'private',
    4: 'ignore',
    5: 'parents',
    6: 'other',
    0: 'unk'
}

df.home = df.home.map(home_values)

marital_values = {
    1: 'single',
    2: 'married',
    3: 'widow',
    4: 'separated',
    5: 'divorced',
    0: 'unk'
}

df.marital = df.marital.map(marital_values)

records_values = {
    1: 'no',
    2: 'yes',
    0: 'unk'
}

df.records = df.records.map(records_values)

job_values = {
    1: 'fixed',
    2: 'partime',
    3: 'freelance',
    4: 'others',
    0: 'unk'
}

df.job = df.job.map(job_values)

for c in ['income', 'assets', 'debt']:
    df[c] = df[c].replace(to_replace=99999999, value=np.nan)

df = df[df.status != 'unk'].reset_index(drop=True)

In [8]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=11)

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = (df_train.status == 'default').astype('int').values
y_test = (df_test.status == 'default').astype('int').values

del df_train['status']
del df_test['status']

In [9]:
dv = DictVectorizer(sparse=False)

train_dicts = df_train.fillna(0).to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

test_dicts = df_test.fillna(0).to_dict(orient='records')
X_test = dv.transform(test_dicts)

In [11]:
dtrain = xgb.DMatrix(X_train, label=y_train)

In [12]:
xgb_params = {
    'eta': 0.1, 
    'max_depth': 3,
    'min_child_weight': 1,

    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=175)

In [13]:
bentoml.xgboost.save_model(
    'credit_risk_model',
    model,
    custom_objects={
        'dict_vectorizer': dv
    }
)

Model(tag="credit_risk_model:uzeb6hsrpct2uico", path="C:\Users\USER\bentoml\models\credit_risk_model\uzeb6hsrpct2uico\")

Size of model after I checked from my filesystem is 119kb

In [21]:
!bentoml models import coolmodel.bentomodel

Error: [models] `import` failed: Item 'mlzoomcamp_homework:qtzdz3slg6mwwdu5' already exists in the store <osfs 'C:\Users\USER\bentoml\models'>


In [23]:
model_ref = bentoml.sklearn.get("mlzoomcamp_homework:qtzdz3slg6mwwdu5")

In [27]:
model_ref.export

<bound method Exportable.export of Model(tag="mlzoomcamp_homework:qtzdz3slg6mwwdu5", path="C:\Users\USER\bentoml\models\mlzoomcamp_homework\qtzdz3slg6mwwdu5")>

scikit-learn version is 1.1.1

 The result to **Question 5** is 
### 1

In [28]:
!bentoml models import coolmodel2.bentomodel

Model(tag="mlzoomcamp_homework:jsi67fslz6txydu5") imported
